# Fixed Effects: Indicator Variables for Groups

One common use of indicator variables are as *fixed effects*. Fixed effects are used when our data as a "nested" structure (we think individual observations belong to groups), and we suspect different things may be happening in each group. 

For example, we might have a dataset of student test scores, and we may suspect that all students at a given school will tend to have higher or lower scores by virtue of being at that school. Or we might have data on earnings and gender, and we may think that incomes are likely to vary across industries or different cities. 

In this regard, fixed effects are analogous in purpose to hierarchical models, though they are slightly different in implementation (differences between fixed effects and hierarchical models are [discussed here](fixed_effects_v_hierarchical.ipynb)). 

In their simplest form, fixed effects can be included in a model just by adding indicator variables for each group. To illustrate, we'll use data from the US Current Population Survey (CPS) on US wages in 2019:

In [1]:
import pandas as pd

# Load survey
cps = pd.read_stata('https://github.com/nickeubank/MIDS_Data/blob/master/Current_Population_Survey/morg18.dta?raw=true')

# Limit to people currently employed and working full time. 
cps = cps[cps.lfsr94 == 'Employed-At Work']
cps = cps[cps.uhourse >= 35]

# And we can adjust earnings per hour (in cents) into dollars, 
cps['earnhre_dollars'] = cps['earnhre'] / 100
cps['annual_earnings'] = cps['earnhre_dollars'] * cps['uhourse'] * 52

# And create gender and college educ variable
cps['female'] = (cps.sex == 2).astype('int')
cps['has_college_educ'] = (cps.grade92 > 43).astype('int')

To illustrate the use of fixed effects, we begin with a simple model of earnings:

In [2]:
import statsmodels.formula.api as smf
smf.ols('annual_earnings ~ female + age + has_college_educ', cps).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     2490.
Date:                Sat, 15 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:00:49   Log-Likelihood:            -7.5063e+05
No. Observations:               65755   AIC:                         1.501e+06
Df Residuals:                   65751   BIC:                         1.501e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         3.239e+04    281.264    115.142      0.000    3.18e+04    3.29e+04
female           -8052.8166    172.035    -46.809      0.000   -8390.004   -7715.629
age                290.0363      6.255     46.370      0.000     277.777     302.296
has_college_educ  2.367e+04    410.579     57.646      0.000    2.29e+04    2.45e+04
==============================================================================
Omnibus:                    31647.447   Durbin-Watson:                   1.882
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           264260.027
Skew:                           2.152   Prob(JB):                         0.00
Kurtosis:                      11.828   Cond. No.                         209.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In this model, we're getting estimates of how education and gender explain variation across all Americans. 

But in this dataset, we also have a variable that tells us the state and county in which each respondent resides. Suppose we think that compensation may vary across regions of the country, and may also be correlated with things like whether a person has a college education. New York, for example, has a strong economy, and more people in NY have college degrees than in the average US city. Grand Forks, North Dakota, by contrast, probably has lower pay and lower education levels. Thus if we don't take into account geography, our estimate of the value of education will likely be overstated since we're capturing not only how education correlates with income, but also the fact that people with college degrees tend to live in towns that pay better. 

To correct for this, we can add an indicator for every county in the US to our model:

In [3]:
# The county variable is an id for counties *within*
# each state, so we have to concatenate county ID numbers
# with state names to get unique values for every actual 
# county in the US. 

cps['county_code'] = cps.stfips + '_county_' + cps.county.astype('str')

In [4]:
smf.ols('annual_earnings ~ female + age + has_college_educ + C(county_code)', cps).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     27.35
Date:                Sat, 15 Feb 2020   Prob (F-statistic):               0.00
Time:                        17:00:52   Log-Likelihood:            -7.4990e+05
No. Observations:               65755   AIC:                         1.500e+06
Df Residuals:                   65423   BIC:                         1.503e+06
Df Model:                         331                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                        3.949e+04    783.626     50.389      0.000     3.8e+04     4.1e+04
C(county_code)[T.AL_county_0]   -9652.4619    981.045     -9.839      0.000   -1.16e+04   -7729.614
C(county_code)[T.AL_county_3]   -8355.2221   2514.351     -3.323      0.001   -1.33e+04   -3427.093
C(county_code)[T.AL_county_81]  -1.089e+04   2798.807     -3.892      0.000   -1.64e+04   -5407.232
C(county_code)[T.AL_county_97]  -8272.5800   2143.347     -3.860      0.000   -1.25e+04   -4071.619
C(county_code)[T.AR_county_0]   -1.088e+04    957.353    -11.360      0.000   -1.28e+04   -8999.196
C(county_code)[T.AZ_county_0]   -1.046e+04   2652.215     -3.945      0.000   -1.57e+04   -5264.623
C(county_code)[T.AZ_county_13]  -7477.3995   1088.083     -6.872      0.000   -9610.042   -5344.757
C(county_code)[T.AZ_county_19]  -9807.0443   1811.956     -5.412      0.000   -1.34e+04   -6255.611
C(county_code)[T.AZ_county_21]  -8487.1774   3000.818     -2.828      0.005   -1.44e+04   -2605.573
C(county_code)[T.AZ_county_25]   2372.8697   3978.441      0.596      0.551   -5424.876    1.02e+04
C(county_code)[T.AZ_county_27]  -8879.7940   2686.601     -3.305      0.001   -1.41e+04   -3614.055
C(county_code)[T.CA_county_0]   -3952.5717    954.582     -4.141      0.000   -5823.552   -2081.591
C(county_code)[T.CA_county_1]    1036.9671   1838.900      0.564      0.573   -2567.277    4641.211
C(county_code)[T.CA_county_107] -1.191e+04   2398.062     -4.967      0.000   -1.66e+04   -7211.288
C(county_code)[T.CA_county_111] -4102.8017   2258.342     -1.817      0.069   -8529.152     323.549
C(county_code)[T.CA_county_113] -8407.7787   4253.392     -1.977      0.048   -1.67e+04     -71.130
C(county_code)[T.CA_county_17]   1.303e+04   4041.889      3.225      0.001    5111.493     2.1e+04
C(county_code)[T.CA_county_19]  -9143.1371   1770.164     -5.165      0.000   -1.26e+04   -5673.615
C(county_code)[T.CA_county_29]  -1.093e+04   2046.674     -5.341      0.000   -1.49e+04   -6919.661
C(county_code)[T.CA_county_31]  -3347.2096   2587.986     -1.293      0.196   -8419.663    1725.244
C(county_code)[T.CA_county_37]  -6544.6140    926.224     -7.066      0.000   -8360.014   -4729.214
C(county_code)[T.CA_county_53]  -6768.1589   3327.560     -2.034      0.042   -1.33e+04    -246.140
C(county_code)[T.CA_county_59]  -7264.8388   1261.099     -5.761      0.000   -9736.594   -4793.084
C(county_code)[T.CA_county_61]   7772.7521   3805.102      2.043      0.041     314.751    1.52e+04
C(county_code)[T.CA_county_67]  -5823.6499   1592.237     -3.658      0.000   -8944.434   -2702.866
C(county_code)[T.CA_county_7]   -2760.6904   3165.378     -0.872      0.383   -8964.832    3443.451
C(county_code)[T.CA_county_73]  -6612.7252   1227.779     -5.386      0.000   -9019

Voila! What you've just estimated is no longer the relationship between education and income across all Americans, but rather the relationship between education and income *within in US county*. 

To be clear, fixed effects aren't really different from adding a normal control variable. One could say that adding `female` means that we're now estimating the relationship between education and income among men and among women. *Mechnically*, fixed effects are just additional indicator variables. But because we often use them for groups, thinking about the fact that, when added, you're effectively no estimating variation *within* the groups specified by the fixed effects is a powerful idea. 

Perhaps no place is this more clear than in full panel data, where you have data on the same entities over time. In a panel regression, the addition of entity fixed effects allow you to difference out any *constant* differences between entities, and focus only on changes within each entity over time. This even works for people! In a panel with individuals observed over time, adding individual fixed effects means you're effectively controlling for anything constant about each individual (things that don't change over time), and now you're just studying *changes over time* for each individual. 

## Clustering

When working with fixed effects, however, it's also often a good idea to cluster your standard errors by your fixed effect variable. Clustering is a method for taking into account some of the variation in your data isn't coming from the individual level (where you have lots of observations), but rather from the group level. Since you have fewer groups than observations, clustering corrects your standard errors to reflect the smaller effective sample size being used to estimate those fixed effects. 

Clustering is thankfully easy to do -- just use the `get_robustcov_results` method from `statsmodels`, and use the `groups` keyword to pass the group assignments for each observation. 

**Note that if you're using formulas in statsmodels, the regression is automatically dropping observations that can't be estimated because of missing data, so you have to do the same before passing your group assignments to** `get_robustcov_results` **-- otherwise you'll get the error:**

```
ValueError: The weights and list don't have the same length.
```

**because the number of observations in the model doesn't match the number of observations in the group assignment vector you pass!**

In [5]:
model = smf.ols('annual_earnings ~ female + age + has_college_educ + C(county_code)', cps).fit()

# Drop any entries with missing data from the model
fe_groups = cps.copy()
for i in ['annual_earnings', 'female', 'age', 'county_code', 'has_college_educ']:
    fe_groups = fe_groups[pd.notnull(fe_groups[i])]

# Adjust SEs
model.get_robustcov_results(cov_type='cluster', groups=fe_groups.county_code).summary()

/Users/Nick/miniconda3/lib/python3.7/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 331, but rank is 3
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        annual_earnings   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     1769.
Date:                Sat, 15 Feb 2020   Prob (F-statistic):          3.93e-202
Time:                        17:00:55   Log-Likelihood:            -7.4990e+05
No. Observations:               65755   AIC:                         1.500e+06
Df Residuals:                   65423   BIC:                         1.503e+06
Df Model:                         331                                         
Covariance Type:              cluster                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                        3.949e+04    350.955    112.510      0.000    3.88e+04    4.02e+04
C(county_code)[T.AL_county_0]   -9652.4619     19.022   -507.447      0.000   -9689.882   -9615.042
C(county_code)[T.AL_county_3]   -8355.2221     13.206   -632.660      0.000   -8381.202   -8329.242
C(county_code)[T.AL_county_81]  -1.089e+04     27.360   -398.135      0.000   -1.09e+04   -1.08e+04
C(county_code)[T.AL_county_97]  -8272.5800     32.360   -255.645      0.000   -8336.239   -8208.921
C(county_code)[T.AR_county_0]   -1.088e+04     19.843   -548.076      0.000   -1.09e+04   -1.08e+04
C(county_code)[T.AZ_county_0]   -1.046e+04      9.993  -1046.982      0.000   -1.05e+04   -1.04e+04
C(county_code)[T.AZ_county_13]  -7477.3995     12.749   -586.531      0.000   -7502.479   -7452.320
C(county_code)[T.AZ_county_19]  -9807.0443     28.932   -338.966      0.000   -9863.960   -9750.128
C(county_code)[T.AZ_county_21]  -8487.1774     45.123   -188.089      0.000   -8575.945   -8398.410
C(county_code)[T.AZ_county_25]   2372.8697     56.684     41.861      0.000    2261.359    2484.380
C(county_code)[T.AZ_county_27]  -8879.7940     45.335   -195.871      0.000   -8968.978   -8790.610
C(county_code)[T.CA_county_0]   -3952.5717      5.989   -659.928      0.000   -3964.354   -3940.789
C(county_code)[T.CA_county_1]    1036.9671     25.627     40.464      0.000     986.554    1087.381
C(county_code)[T.CA_county_107] -1.191e+04     21.069   -565.359      0.000    -1.2e+04   -1.19e+04
C(county_code)[T.CA_county_111] -4102.8017     28.583   -143.537      0.000   -4159.032   -4046.572
C(county_code)[T.CA_county_113] -8407.7787     51.759   -162.441      0.000   -8509.600   -8305.958
C(county_code)[T.CA_county_17]   1.303e+04     47.262    275.775      0.000    1.29e+04    1.31e+04
C(county_code)[T.CA_county_19]  -9143.1371     25.627   -356.777      0.000   -9193.551   -9092.723
C(county_code)[T.CA_county_29]  -1.093e+04     23.250   -470.150      0.000    -1.1e+04   -1.09e+04
C(county_code)[T.CA_county_31]  -3347.2096     35.181    -95.141      0.000   -3416.419   -3278.000
C(county_code)[T.CA_county_37]  -6544.6140      9.595   -682.069      0.000   -6563.490   -6525.738
C(county_code)[T.CA_county_53]  -6768.1589     38.391   -176.293      0.000   -6843.683   -6692.634
C(county_code)[T.CA_county_59]  -7264.8388     10.096   -719.579      0.000   -7284.700   -7244.978
C(county_code)[T.CA_county_61]   7772.7521     44.901    173.108      0.000    7684.422    7861.083
C(county_code)[T.CA_county_67]  -5823.6499     23.314   -249.792      0.000   -5869.514   -5777.786
C(county_code)[T.CA_county_7]   -2760.6904     24.738   -111.597      0.000   -2809.355   -2712.025
C(county_code)[T.CA_county_73]  -6612.7252      4.797  -1378.460      0.000   -6622

## PanelOLS

OK, so everything we've describe up till here is a reasonable approach to fixed effects, but it has two limitations: our regression output looks *terrible*, and computing all those intercepts was slow.

This brings us to some of the specialized methods for calculating fixed effects. It turns out that if you aren't interested in the coefficient on each fixed effect, there are much more computationally efficient methods of calculating fixed effects. But to use them, we'll have to use a different library: [linearmodels](https://bashtage.github.io/linearmodels/doc/index.html) (installable using `pip install linearmodels`).

In particular, we'll be using the `PanelOLS` function from `linearmodels`. As the name implies, `PanelOLS` is designed for linear regression (social scientists call linear regression Ordinary Least Squares, or OLS) with panel data, which is really any form of data organized along two dimensions. Normally a panel has data on many entities observed several times, so the first dimension is the `entity` dimension, and the second is the `time` dimension. 

In this case, we don't really have a panel -- just nested data -- but because fixed effects are commonly used in panels, we'll use this tool. 

The only catch is: you have to use `multiindexes` in `pandas`. I *know*, I hate them too. But the multi-index is required by the library for it to understand what variable constitutes the "group" for which you want to add fixed effects. Basically `PanelOLS` calls the first level of the multi-index the `entity` and the second level `time`. In this case, though, we'll just make the first level our counties, and the second level individual identifiers, then use `entity` fixed effects (and clustering). 

In [6]:
cps.head()

county  smsastat  age  sex  grade92  race  ethnic  marital  uhourse  \
2        0       1.0   52    2       39     2     NaN        5     40.0   
3        0       1.0   19    2       39     2     NaN        7     40.0   
4        0       1.0   56    2       43     2     NaN        5     40.0   
6       97       1.0   48    1       39     1     NaN        7     40.0   
17      97       1.0   59    1       39     2     NaN        7     40.0   

    earnhre  ...            lfsr94               class94  unioncov  \
2    2084.0  ...  Employed-At Work    Government - State             
3    1000.0  ...  Employed-At Work   Private, For Profit        No   
4    2500.0  ...  Employed-At Work  Government - Federal             
6    1700.0  ...  Employed-At Work   Private, For Profit        No   
17   2000.0  ...  Employed-At Work   Private, For Profit        No   

                                                ind02  stfips  \
2   Residential care facilities, without nursing (...      AL   
3                    Business support services (5614)      AL   
4                                     Hospitals (622)      AL   
6                          Truck transportation (484)      AL   
17  ****Department stores and discount stores (s45...      AL   

    earnhre_dollars  annual_earnings  female  has_college_educ   county_code  
2             20.84          43347.2       1                 0   AL_county_0  
3             10.00          20800.0       1                 0   AL_county_0  
4             25.00          52000.0       1                 0   AL_county_0  
6             17.00          35360.0       0                 0  AL_county_97  
17            20.00          41600.0       0                 0  AL_county_97  

[5 rows x 30 columns]

In [7]:
# Move county groups into highest level of multi-index, 
# with old index in second level. 
# PanelOLS will then see the first level as the `entity` 
# identifier. 
cps = cps.set_index(['county_code', cps.index])

In [8]:
from linearmodels import PanelOLS
mod = PanelOLS.from_formula('annual_earnings ~ female + age + has_college_educ + EntityEffects', data=cps)
mod.fit(cov_type='clustered', cluster_entity=True)

/Users/Nick/miniconda3/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


PanelOLS Estimation Summary                           
================================================================================
Dep. Variable:        annual_earnings   R-squared:                        0.1000
Estimator:                   PanelOLS   R-squared (Between):              0.3898
No. Observations:               65755   R-squared (Within):               0.1000
Date:                Sat, Feb 15 2020   R-squared (Overall):              0.3265
Time:                        17:00:55   Log-likelihood                -7.499e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2422.5
Entities:                         329   P-value                           0.0000
Avg Obs:                       199.86   Distribution:                 F(3,65423)
Min Obs:                       2.0000                                           
Max Obs:                       3219.0   F-statistic (robust):             937.96
                                        P-value                           0.0000
Time periods:                   65755   Distribution:                 F(3,65423)
Avg Obs:                       1.0000                                           
Min Obs:                       1.0000                                           
Max Obs:                       1.0000                                           
                                                                                
                                Parameter Estimates                                 
====================================================================================
                  Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------
female              -7923.2     272.07    -29.122     0.0000     -8456.4     -7389.9
age                  287.28     7.6942     37.336     0.0000      272.20      302.36
has_college_educ  2.313e+04     768.88     30.086     0.0000   2.163e+04   2.464e+04
====================================================================================

F-test for Poolability: 4.4314
P-value: 0.0000
Distribution: F(328,65423)

Included effects: Entity
PanelEffectsResults, id: 0x7fe1514b06d8

As you can see, the coefficients of the `PanelOLS` model are exactly the same as those we calculated above, and the standard errors are nearly identical (there are a few ways to calculate clustered standard errors, so they aren't numerically equivalent). But the way `PanelOLS` added fixed effects was much more computationally efficient efficient, and in these situations, we don't usually want to see the coefficients, so they're suppressed. 